In [87]:
import torchvision
from torchvision import transforms
import torch
import os
import numpy as np
from torch import nn
from collections import OrderedDict
from torch.autograd import Variable
import torch.nn.functional as F
import pandas as pd
import torchvision.models as models
import os
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多
#DEVICE = torch.device("cpu") 

import warnings
warnings.filterwarnings('ignore')#有时候运行代码时会有很多warning输出，如提醒新版本之类的，如果不想这些乱糟糟的输出可以这样：

In [88]:
model = models.resnet50(pretrained=True)#加载预训练的模型，也可以不加载

In [89]:
print(model)#查看renet50的网络结构，如果不修改网络结构来训练自己的模型的话，只需要修改全连接层的输入和输出即可

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [90]:
print(model.fc)

Linear(in_features=2048, out_features=1000, bias=True)


In [91]:
#model.fc = nn.Linear(2048,33)#只修改最后一层全连接模型，由于我们昆虫数据集只有33个分类


#或者这种写法
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 17)



In [92]:
print(model)#查看模型是否被成果修改

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [93]:
#把模型指定在GPU上运行
#model = model.to(DEVICE)
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [94]:
#读取数据，并对数据进行归一化处理
os.getcwd()

'/data0/zengpz'

In [95]:
picturepath = '/data0/zengpz/damage'


In [96]:
import os.path
print(os.path.basename(picturepath))
print(os.path.dirname(picturepath))

import glob

print(glob.glob('/data0/zengpz/damage/*'))

damage
/data0/zengpz
['/data0/zengpz/damage/黑斑病', '/data0/zengpz/damage/叶斑病', '/data0/zengpz/damage/蛴螬', '/data0/zengpz/damage/软腐病', '/data0/zengpz/damage/炭疽病', '/data0/zengpz/damage/煤污病', '/data0/zengpz/damage/叶枯病', '/data0/zengpz/damage/流胶病', '/data0/zengpz/damage/蛾', '/data0/zengpz/damage/褐斑病', '/data0/zengpz/damage/介壳虫', '/data0/zengpz/damage/白粉病', '/data0/zengpz/damage/螨', '/data0/zengpz/damage/根腐病', '/data0/zengpz/damage/蚜', '/data0/zengpz/damage/飞虱', '/data0/zengpz/damage/马陆']


In [97]:
list_dir = [i for i in glob.glob('/data0/zengpz/damage/*')]
print(len(list_dir))
#show其中一张图片的路径
list_dir_ =[]
for i in range(len(list_dir)):
    list_dir_ .append(list_dir[i].split('/')[4])
print(list_dir_)
print(len(list_dir_))#可以看到一共17个类

17
['黑斑病', '叶斑病', '蛴螬', '软腐病', '炭疽病', '煤污病', '叶枯病', '流胶病', '蛾', '褐斑病', '介壳虫', '白粉病', '螨', '根腐病', '蚜', '飞虱', '马陆']
17


In [98]:
from PIL import Image
#files = sorted(glob.glob('/data0/zengpz/insect（我处理）/桑褐刺蛾/*.jpg'))

files = sorted(glob.glob('/data0/zengpz/damage/*/*.jpg'))

#print(files[32])
print(len(files))#一共3400张照片
#print(files)
#t = '/data0/zengpz/insect（我处理）/桑褐刺蛾/1.jpg'
#t in files


3162


In [99]:
#查看显示某一张图片
from PIL import Image
files = sorted(glob.glob('/data0/zengpz/damage/*/*.jpg'))
#im = Image.open(r'list_dir[0]/')
#im.show()
image_labels = []
filespath =[]
for file in files:  
    image_labels.append(file.split('/')[4])
    filespath.append(file)
    
print(len(filespath))#一共3400张照片
print(len(image_labels))

#把路径和label存成dataframe的形式
from pandas import Series,DataFrame
import pandas as pd

df = DataFrame()
df['chonghai_dir'] = filespath
df['chonghai_label'] = image_labels

print(df.head())

#保存成csv文件，便于之后使用
#
#df.to_csv('damage_dataread.csv')

3162
3162
                       chonghai_dir chonghai_label
0    /data0/zengpz/damage/介壳虫/1.jpg            介壳虫
1   /data0/zengpz/damage/介壳虫/10.jpg            介壳虫
2  /data0/zengpz/damage/介壳虫/100.jpg            介壳虫
3  /data0/zengpz/damage/介壳虫/101.jpg            介壳虫
4  /data0/zengpz/damage/介壳虫/102.jpg            介壳虫


In [100]:
#把数据随机按照比例1:10划分训练集和测试集
test_index = range(0,3162,10) 
train_index = []

for i in range(0,3162):
    if i not in test_index:
        train_index.append(i)

train = df.iloc[train_index,:]
test = df.iloc[test_index,:]

print(len(train))
print(len(test))
#保存train数据
#train.to_csv('damage_train.csv')
#test.to_csv('damage_test.csv')

2845
317


In [101]:
#显示某张图片看一下
import cv2
import numpy as np
img = cv2.imread(df['chonghai_dir'][3])
print(img.shape)


(667, 500, 3)


In [102]:
#对原始图片的预处理
from torchvision import transforms as transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.RandomRotation((-45,45)), #随机旋转
    transforms.ToTensor()
    #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.229, 0.224, 0.225)), #R,G,B每层的归一化用到的均值和方差
])

In [103]:
from torch.utils.data import Dataset
from PIL import Image


#定义一个数据集
class myDataset(Dataset):
    """ 数据集演示 """
    def __init__(self, csv_file):
        """实现初始化方法，在初始化的时候将数据读载入"""
        self.df=pd.read_csv(csv_file)
        self.tags = {'黑斑病':0, '叶斑病':1, '蛴螬':2, '软腐病':3, '炭疽病':4, '煤污病':5,
                      '叶枯病':6, '流胶病':7, '蛾':8, '褐斑病':9, '介壳虫':10, '白粉病':11,
                      '螨':12, '根腐病':13, '蚜':14, '飞虱':15, '马陆':16}
        
    def __len__(self):
        '''
        返回df的长度
        '''
        return len(self.df)
    def __getitem__(self, idx):
        '''
        根据 idx 返回一行数据
        
        '''
        img = Image.open(self.df.iloc[idx].chonghai_dir).convert('RGB')
        label = self.df.iloc[idx].chonghai_label
        label = self.tags[label]
        img_path = self.df.iloc[idx].chonghai_dir
        #把图片做归一化处理
        img = transform(img)
        return img,label,img_path

#修改读取的文件的路径    
#ds_demo= myDataset('insect_dataread.csv')
train_data =  myDataset('damage_train.csv')
test_data = myDataset('damage_test.csv')
print(len(train_data))
print(len(test_data))

2845
317


In [104]:
train_data[1]#用索引可以直接访问对应的数据, 对应 __getitem__ 方法

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 10,
 '/data0/zengpz/damage/介壳虫/100.jpg')

In [105]:
train_data[1][0].shape  #可以看到输入的图片已经做了归一化处理了

torch.Size([3, 224, 224])

In [106]:
#测试一下是否真的有17类数据来训练
'''
list_ = []
for i in range(0,2845):
    list_.append(train_data[i][1])
    if train_data[i][1] < 0 or train_data[i][1] > 17:
        print(train_data[i][2])
print(len(set(list_)))
set(list_)
'''

'\nlist_ = []\nfor i in range(0,2845):\n    list_.append(train_data[i][1])\n    if train_data[i][1] < 0 or train_data[i][1] > 17:\n        print(train_data[i][2])\nprint(len(set(list_)))\nset(list_)\n'

In [107]:

#使用官方提供的数据载入器，读取数据
#常用参数有：batch_size(每个batch的大小), shuffle(是否进行shuffle操作), num_workers(加载数据的时候使用几个子进程)
train_ = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True, num_workers=0)
test_ = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True, num_workers=0)

#DataLoader返回的是一个可迭代对象，我们可以使用迭代器分次获取数据
#idata=iter(train)
#print(next(idata))

In [108]:

#loss function and optimizer
criterion=torch.nn.CrossEntropyLoss()
#parameters only train the last fc layer
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)


In [109]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def trainmodel(model,train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (inputs_, labels_, _) in enumerate(train_loader):
    #for batch_idx, (data, target) in enumerate(train_loader):
        #data, target = data.to(device), target.to(device)
        
        #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        #inputs = inputs.to(device)#用GPU训练
        #labels = labels.to(device)
        
        
        cuda = torch.device('cuda') 
        #直接用torch.tensor会有warning
        inputs = torch.tensor(inputs_).cuda()
        labels = torch.tensor(labels_).cuda()
        #inputs = inputs_.clone().detach().requires_grad_(True)
        #labels = labels_.clone().detach()
        
        
        #inputs = inputs.to(device)#用GPU训练
        #labels = labels.to(device)
        #inputs = inputs.cpu()
        #labels = labels.cpu()
        
        
        optimizer.zero_grad()
        output = model(inputs)
        
        #loss = F.nll_loss(output, target)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(inputs), len(train_loader)*len(inputs),
                100. * batch_idx / len(train_loader), loss.item()))
    

In [112]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
accuracy = []
def testmodel(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (inputs, labels, _) in enumerate(test_loader):
            
            cuda = torch.device('cuda') 
            #这样有warning
            data = torch.tensor(inputs).cuda()
            target = torch.tensor(labels).cuda()
            #data = inputs.clone().detach().requires_grad_(True)
            #target = labels.clone().detach
            
            
            #device = torch.device("cuda")
            #data, target = data.to(device), target.to(device)
            #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            
            data, target = data.to(device), target.to(device)
            #data, target = data.cpu(), target.cpu()
            
            output = model(data)
            test_loss += criterion(output, target).item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    accuracy.append(100. * correct / (len(inputs)*len(test_loader)))
    if 100. * correct / (len(inputs)*len(test_loader)) >= max(accuracy):
        torch.save(model.state_dict,'resnet50_chonghai_parameters.pkl')
    test_loss /= len(inputs)*len(test_loader)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(inputs)*len(test_loader),
        100. * correct / (len(inputs)*len(test_loader))))



In [113]:
#开始训练过程
EPOCHS = 100
for epoch in range(1, EPOCHS + 1):
    trainmodel(model, train_, optimizer, epoch)    
    testmodel(model,test_)
print('最好的识别率为：%f'%(max(t_result)))

Train Epoch: 1 [232/2848 (8%)]	Loss: 2.449261
Train Epoch: 1 [472/2848 (17%)]	Loss: 2.630345
Train Epoch: 1 [712/2848 (25%)]	Loss: 2.864708
Train Epoch: 1 [952/2848 (33%)]	Loss: 2.527807
Train Epoch: 1 [1192/2848 (42%)]	Loss: 2.587501
Train Epoch: 1 [1432/2848 (50%)]	Loss: 2.872996
Train Epoch: 1 [1672/2848 (59%)]	Loss: 2.821906
Train Epoch: 1 [1912/2848 (67%)]	Loss: 2.780685
Train Epoch: 1 [2152/2848 (76%)]	Loss: 2.985184
Train Epoch: 1 [2392/2848 (84%)]	Loss: 2.985437
Train Epoch: 1 [2632/2848 (92%)]	Loss: 2.475358

Test set: Average loss: 0.5219, Accuracy: 33/200 (16%)

Train Epoch: 2 [232/2848 (8%)]	Loss: 2.737757
Train Epoch: 2 [472/2848 (17%)]	Loss: 2.811751
Train Epoch: 2 [712/2848 (25%)]	Loss: 2.810771
Train Epoch: 2 [952/2848 (33%)]	Loss: 2.610869
Train Epoch: 2 [1192/2848 (42%)]	Loss: 2.291857
Train Epoch: 2 [1432/2848 (50%)]	Loss: 2.555051
Train Epoch: 2 [1672/2848 (59%)]	Loss: 2.310217
Train Epoch: 2 [1912/2848 (67%)]	Loss: 2.640107
Train Epoch: 2 [2152/2848 (76%)]	Loss: 2.

Train Epoch: 15 [472/2848 (17%)]	Loss: 2.539129
Train Epoch: 15 [712/2848 (25%)]	Loss: 2.283441
Train Epoch: 15 [952/2848 (33%)]	Loss: 2.770785
Train Epoch: 15 [1192/2848 (42%)]	Loss: 2.204360
Train Epoch: 15 [1432/2848 (50%)]	Loss: 2.590899
Train Epoch: 15 [1672/2848 (59%)]	Loss: 2.293252
Train Epoch: 15 [1912/2848 (67%)]	Loss: 1.795936
Train Epoch: 15 [2152/2848 (76%)]	Loss: 2.144001
Train Epoch: 15 [2392/2848 (84%)]	Loss: 2.105593
Train Epoch: 15 [2632/2848 (92%)]	Loss: 2.003122

Test set: Average loss: 0.4616, Accuracy: 83/200 (42%)

Train Epoch: 16 [232/2848 (8%)]	Loss: 2.222526
Train Epoch: 16 [472/2848 (17%)]	Loss: 2.504892
Train Epoch: 16 [712/2848 (25%)]	Loss: 3.015508
Train Epoch: 16 [952/2848 (33%)]	Loss: 2.350688
Train Epoch: 16 [1192/2848 (42%)]	Loss: 1.758150
Train Epoch: 16 [1432/2848 (50%)]	Loss: 1.925414
Train Epoch: 16 [1672/2848 (59%)]	Loss: 2.395375
Train Epoch: 16 [1912/2848 (67%)]	Loss: 1.936930
Train Epoch: 16 [2152/2848 (76%)]	Loss: 3.065396
Train Epoch: 16 [239

Train Epoch: 29 [232/2848 (8%)]	Loss: 1.604497
Train Epoch: 29 [472/2848 (17%)]	Loss: 1.828484
Train Epoch: 29 [712/2848 (25%)]	Loss: 2.254837
Train Epoch: 29 [952/2848 (33%)]	Loss: 1.899946
Train Epoch: 29 [1192/2848 (42%)]	Loss: 1.664507
Train Epoch: 29 [1432/2848 (50%)]	Loss: 1.857858
Train Epoch: 29 [1672/2848 (59%)]	Loss: 1.788832
Train Epoch: 29 [1912/2848 (67%)]	Loss: 1.544524
Train Epoch: 29 [2152/2848 (76%)]	Loss: 3.010834
Train Epoch: 29 [2392/2848 (84%)]	Loss: 1.698916
Train Epoch: 29 [2632/2848 (92%)]	Loss: 1.969468

Test set: Average loss: 0.5553, Accuracy: 90/200 (45%)

Train Epoch: 30 [232/2848 (8%)]	Loss: 1.784714
Train Epoch: 30 [472/2848 (17%)]	Loss: 1.063007
Train Epoch: 30 [712/2848 (25%)]	Loss: 1.774217
Train Epoch: 30 [952/2848 (33%)]	Loss: 2.122136
Train Epoch: 30 [1192/2848 (42%)]	Loss: 1.658652
Train Epoch: 30 [1432/2848 (50%)]	Loss: 1.685480
Train Epoch: 30 [1672/2848 (59%)]	Loss: 1.679894
Train Epoch: 30 [1912/2848 (67%)]	Loss: 1.311632
Train Epoch: 30 [2152/


Test set: Average loss: 0.5341, Accuracy: 112/200 (56%)

Train Epoch: 43 [232/2848 (8%)]	Loss: 2.253450
Train Epoch: 43 [472/2848 (17%)]	Loss: 1.045686
Train Epoch: 43 [712/2848 (25%)]	Loss: 1.051376
Train Epoch: 43 [952/2848 (33%)]	Loss: 1.420443
Train Epoch: 43 [1192/2848 (42%)]	Loss: 0.932774
Train Epoch: 43 [1432/2848 (50%)]	Loss: 0.749734
Train Epoch: 43 [1672/2848 (59%)]	Loss: 0.924905
Train Epoch: 43 [1912/2848 (67%)]	Loss: 1.345502
Train Epoch: 43 [2152/2848 (76%)]	Loss: 1.388630
Train Epoch: 43 [2392/2848 (84%)]	Loss: 1.044001
Train Epoch: 43 [2632/2848 (92%)]	Loss: 1.865926

Test set: Average loss: 0.5118, Accuracy: 126/200 (63%)

Train Epoch: 44 [232/2848 (8%)]	Loss: 1.217006
Train Epoch: 44 [472/2848 (17%)]	Loss: 1.294876
Train Epoch: 44 [712/2848 (25%)]	Loss: 0.794219
Train Epoch: 44 [952/2848 (33%)]	Loss: 1.282965
Train Epoch: 44 [1192/2848 (42%)]	Loss: 0.786331
Train Epoch: 44 [1432/2848 (50%)]	Loss: 2.038388
Train Epoch: 44 [1672/2848 (59%)]	Loss: 0.945497
Train Epoch:

Train Epoch: 56 [2632/2848 (92%)]	Loss: 0.957933

Test set: Average loss: 0.3815, Accuracy: 166/200 (83%)

Train Epoch: 57 [232/2848 (8%)]	Loss: 0.844749
Train Epoch: 57 [472/2848 (17%)]	Loss: 0.623440
Train Epoch: 57 [712/2848 (25%)]	Loss: 0.871789
Train Epoch: 57 [952/2848 (33%)]	Loss: 2.078491
Train Epoch: 57 [1192/2848 (42%)]	Loss: 1.245357
Train Epoch: 57 [1432/2848 (50%)]	Loss: 0.685629
Train Epoch: 57 [1672/2848 (59%)]	Loss: 1.450398
Train Epoch: 57 [1912/2848 (67%)]	Loss: 0.538629
Train Epoch: 57 [2152/2848 (76%)]	Loss: 1.009951
Train Epoch: 57 [2392/2848 (84%)]	Loss: 0.596832
Train Epoch: 57 [2632/2848 (92%)]	Loss: 0.567325

Test set: Average loss: 0.3704, Accuracy: 177/200 (88%)

Train Epoch: 58 [232/2848 (8%)]	Loss: 1.507743
Train Epoch: 58 [472/2848 (17%)]	Loss: 0.578487
Train Epoch: 58 [712/2848 (25%)]	Loss: 0.610558
Train Epoch: 58 [952/2848 (33%)]	Loss: 0.904988
Train Epoch: 58 [1192/2848 (42%)]	Loss: 1.342466
Train Epoch: 58 [1432/2848 (50%)]	Loss: 0.549539
Train Epoch:

Train Epoch: 70 [2392/2848 (84%)]	Loss: 0.562734
Train Epoch: 70 [2632/2848 (92%)]	Loss: 0.887148

Test set: Average loss: 0.4867, Accuracy: 174/200 (87%)

Train Epoch: 71 [232/2848 (8%)]	Loss: 0.398502
Train Epoch: 71 [472/2848 (17%)]	Loss: 0.115658
Train Epoch: 71 [712/2848 (25%)]	Loss: 0.249168
Train Epoch: 71 [952/2848 (33%)]	Loss: 0.514450
Train Epoch: 71 [1192/2848 (42%)]	Loss: 1.275263
Train Epoch: 71 [1432/2848 (50%)]	Loss: 0.849116
Train Epoch: 71 [1672/2848 (59%)]	Loss: 0.432839
Train Epoch: 71 [1912/2848 (67%)]	Loss: 0.267925
Train Epoch: 71 [2152/2848 (76%)]	Loss: 0.121595
Train Epoch: 71 [2392/2848 (84%)]	Loss: 0.378206
Train Epoch: 71 [2632/2848 (92%)]	Loss: 0.090384

Test set: Average loss: 0.3790, Accuracy: 193/200 (96%)

Train Epoch: 72 [232/2848 (8%)]	Loss: 0.257447
Train Epoch: 72 [472/2848 (17%)]	Loss: 0.576310
Train Epoch: 72 [712/2848 (25%)]	Loss: 1.488587
Train Epoch: 72 [952/2848 (33%)]	Loss: 0.837183
Train Epoch: 72 [1192/2848 (42%)]	Loss: 0.283509
Train Epoch:

Train Epoch: 84 [1912/2848 (67%)]	Loss: 0.609021
Train Epoch: 84 [2152/2848 (76%)]	Loss: 0.289151
Train Epoch: 84 [2392/2848 (84%)]	Loss: 0.330248
Train Epoch: 84 [2632/2848 (92%)]	Loss: 0.262304

Test set: Average loss: 0.3199, Accuracy: 229/200 (114%)

Train Epoch: 85 [232/2848 (8%)]	Loss: 0.499249
Train Epoch: 85 [472/2848 (17%)]	Loss: 0.814739
Train Epoch: 85 [712/2848 (25%)]	Loss: 0.282914
Train Epoch: 85 [952/2848 (33%)]	Loss: 0.246319
Train Epoch: 85 [1192/2848 (42%)]	Loss: 0.490537
Train Epoch: 85 [1432/2848 (50%)]	Loss: 0.767853
Train Epoch: 85 [1672/2848 (59%)]	Loss: 0.235877
Train Epoch: 85 [1912/2848 (67%)]	Loss: 0.121014
Train Epoch: 85 [2152/2848 (76%)]	Loss: 0.179683
Train Epoch: 85 [2392/2848 (84%)]	Loss: 0.288470
Train Epoch: 85 [2632/2848 (92%)]	Loss: 0.318012

Test set: Average loss: 0.3840, Accuracy: 204/200 (102%)

Train Epoch: 86 [232/2848 (8%)]	Loss: 0.335878
Train Epoch: 86 [472/2848 (17%)]	Loss: 0.101009
Train Epoch: 86 [712/2848 (25%)]	Loss: 0.407774
Train Epo

Train Epoch: 98 [1432/2848 (50%)]	Loss: 0.315015
Train Epoch: 98 [1672/2848 (59%)]	Loss: 0.345588
Train Epoch: 98 [1912/2848 (67%)]	Loss: 0.326368
Train Epoch: 98 [2152/2848 (76%)]	Loss: 0.149024
Train Epoch: 98 [2392/2848 (84%)]	Loss: 0.210119
Train Epoch: 98 [2632/2848 (92%)]	Loss: 0.096005

Test set: Average loss: 0.3439, Accuracy: 230/200 (115%)

Train Epoch: 99 [232/2848 (8%)]	Loss: 0.184578
Train Epoch: 99 [472/2848 (17%)]	Loss: 0.189385
Train Epoch: 99 [712/2848 (25%)]	Loss: 0.040343
Train Epoch: 99 [952/2848 (33%)]	Loss: 1.459046
Train Epoch: 99 [1192/2848 (42%)]	Loss: 0.348731
Train Epoch: 99 [1432/2848 (50%)]	Loss: 0.128946
Train Epoch: 99 [1672/2848 (59%)]	Loss: 0.059888
Train Epoch: 99 [1912/2848 (67%)]	Loss: 0.490520
Train Epoch: 99 [2152/2848 (76%)]	Loss: 0.207266
Train Epoch: 99 [2392/2848 (84%)]	Loss: 0.312900
Train Epoch: 99 [2632/2848 (92%)]	Loss: 0.084442

Test set: Average loss: 0.3839, Accuracy: 188/200 (94%)

Train Epoch: 100 [232/2848 (8%)]	Loss: 0.613567
Train E

ValueError: max() arg is an empty sequence

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure
plt.plot(range(0,100),accuracy)
plt.xlabel('epoch')
plt.ylabel('accuracy')

In [61]:
#随机读入一张照片，输出预测的结果

from PIL import Image
#读入图片
img = Image.open('/data0/zengpz/damage/软腐病/1.jpg')  
input=transform(img)#这里经过转换后输出的input格式是[C,H,W],网络输入还需要增加一维批量大小B
img = torch.tensor(input).cuda()
img = img.unsqueeze(0)#增加一维，输出的img格式为[1,C,H,W],这是因为model的输入是【batch，channel，H,W】
 


model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 17)
model.load_state_dict(torch.load('chonghai_params.pkl'))
model.cuda()
 
res = model(img)#将图片输入网络得到输出
print(res.shape)  #torch.Size([1, 33])
res.max(1)#按列取最大值
res.max(1)[1] #取出最大值的下标对应的index，也就是对应的分类的label

#max_value,index = t.max(probability,1)#找到最大概率对应的索引号，该图片即为该索引号对应的类别
#print(index)


/home/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


torch.Size([1, 17])


tensor([3], device='cuda:0')